<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- The basics of image segmentation

This notebook builds on the previous tutorials for Astropy Units/Quantities, Coordinates, FITS, and Tables.
</div>

## Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from photutils.utils import calc_total_error

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

data = sci_hdulist[0].data.astype(float)
error = rms_hdulist[0].data.astype(float)
hdr = sci_hdulist[0].header
wcs = WCS(hdr)

eff_gain = hdr['TEXPTIME']  # exposure time from the FITS header
total_error = calc_total_error(data, error, eff_gain)

In [ ]:
from astropy.visualization import simple_norm

plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
plt.title('XDF F160W Cutout');

# Image Segmentation

Image segmentation is a process of assigning a label to every pixel in an image such that pixels with the same label are part of the same source. Detected sources must have a minimum number of connected pixels that are each greater than a specified threshold value in an image.

The threshold level is usually defined as some multiple of the background noise (sigma level) above the background. This can be specified either as a per-pixel threshold image or a single value for the whole image.

The image is usually filtered before thresholding to smooth the noise and maximize the detectability of objects with a shape similar to the filter kernel.

First, we need to subtract the background from the image. Even though the background has already been subtracted from this dataset, we'll still perform that step as an example for other datasets.

In this example, we’ll use the `Background2D` class to produce a background and background noise image.  The background rms will be used later to define the detection threshold.

In [ ]:
from photutils.background import Background2D, MedianBackground

bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3),
                   bkg_estimator=bkg_estimator)
data -= bkg.background  # subtract the background

After subtracting the background, we need to define the detection threshold. In this example, we’ll define a 2D detection threshold image using the background RMS image calculate above. We set the threshold at the 2.0$\sigma$ (per pixel) noise level (above the background).

In [ ]:
threshold = 2.0 * bkg.background_rms

Next, let’s convolve the background-subtracted data with a 2D Gaussian kernel with a FWHM of 3 pixels.

In [ ]:
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel

kernel = make_2dgaussian_kernel(3.0, size=9)  # FWHM = 3.0
convolved_data = convolve(data, kernel)

Now we are ready to detect the sources in the background-subtracted convolved image. Let’s find sources that have 5 connected pixels that are each greater than the corresponding pixel-wise threshold level defined above (i.e., 2.0$\sigma$ per pixel above the background). For this we use the [detect_sources](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.detect_sources.html) function.

Note that by default “connected pixels” means “8-connected” pixels, where pixels touch along their edges or corners. One can also use “4-connected” pixels that touch only along their edges by setting connectivity=4:

"8-connected" pixels touch along their edges or corners. "4-connected" pixels touch along their edges. For reference, SourceExtractor uses "8-connected" pixels.

```
8-connected      4-connected
(default)

  1 1 1            0 1 0
  1 x 1            1 x 1
  1 1 1            0 1 0
```

In [ ]:
from photutils.segmentation import detect_sources

npixels = 5
segment_map = detect_sources(convolved_data, threshold, npixels)

print('Found {0} sources'.format(segment_map.nlabels))

The result is a segmentation image ([SegmentationImage](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SegmentationImage.html#photutils.segmentation.SegmentationImage) object).  The segmentation image is an array with the same size as the science image in which each detected source is labeled with a unique integer value (>= 1).  Background pixels have a value of 0.  As a simple example, a segmentation map containing two distinct sources (labeled 1 and 2) might look like this:

```
0 0 0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0 0 0
1 1 1 1 1 0 0 0 2 0
1 1 1 1 0 0 0 2 2 2
1 1 1 0 0 0 2 2 2 2
1 1 1 1 0 0 0 2 2 0
1 1 0 0 0 0 2 2 0 0
0 1 0 0 0 0 2 0 0 0
0 0 0 0 0 0 0 0 0 0
```
where all of the pixels labeled `1` belong to the first source, all those labeled `2` belong to the second, and all those labeled `0` are background pixels.

Let's display the segmentation image.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10))
ax1.imshow(data, norm=norm)
lbl1 = ax1.set_title('Data')
segment_map.imshow(ax=ax2)
lbl2 = ax2.set_title('Segmentation Image')

## Source deblending

Comparing the data array with the segmentation image, we see that several detected sources were blended together.  We can deblend them using the [deblend_sources](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.deblend_sources.html#photutils.segmentation.deblend_sources) function, which uses a combination of multi-thresholding and watershed segmentation.

The amount of deblending can be controlled with the two `deblend_sources` keywords `nlevels` and `contrast`:

- `nlevels` is the number of multi-thresholding levels to use
- `contrast` is the fraction of the total source flux that a local peak must have to be considered as a separate object

In [ ]:
from photutils.segmentation import deblend_sources

segment_map2 = deblend_sources(convolved_data, segment_map, npixels,
                               contrast=0.001, nlevels=32, progress_bar=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 8))
ax1.imshow(data, norm=norm)
ax1.set_title('Data')
segment_map.imshow(ax=ax2)
ax2.set_title('Original Segmentation Image')
segment_map2.imshow(ax=ax3)
ax3.set_title('Deblended Segmentation Image')

print('Found {0} sources'.format(segment_map2.nlabels))

From the segmentation image, we observe that the sources were successfully deblended. We have 6 additional unique sources.

## SourceFinder class

The `SourceFinder` class is a convenience class that combines the functionality of `detect_sources` and `deblend_sources`. After defining the `SourceFinder` object with the desired detection and deblending parameters, you call it with the background-subtracted (convolved) image and threshold.

In [ ]:
from photutils.segmentation import SourceFinder

finder = SourceFinder(npixels=npixels, deblend=True, progress_bar=False)
segment_map3 = finder(convolved_data, threshold)
axim = segment_map3.imshow()

## Modifying a Segmentation Image

The `SegmentationImage` object provides several methods that can be used to visualize or modify itself (e.g., combining labels, removing labels, removing border segments) prior to measuring source photometry and other source properties, including:

* `reassign_label()`: Reassign one or more label numbers.

* `relabel_consecutive()`: Reassign the label numbers consecutively, such that there are no missing label numbers.

* `keep_labels()`: Keep only the specified labels.

* `remove_labels()`: Remove one or more labels.

* `remove_border_labels()`: Remove labeled segments near the image border.

* `remove_masked_labels()`: Remove labeled segments located within a masked region.

* `outline_segments()`: Outline the labeled segments for plotting.

## Photometry, Centroids, and Shape Properties

The [SourceCatalog](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SourceCatalog.html#photutils.segmentation.SourceCatalog) class is the primary tool for measuring the photometry, centroids, and shape/morphological properties of sources defined in a segmentation image. In its most basic form, it takes as input the (background-subtracted) image and the segmentation image. Usually the convolved image is also input, from which the source centroids and shape/morphological properties are measured (if not input, the unconvolved image is used instead).

Let’s continue our example from above and measure the properties of the detected sources.

In [ ]:
import astropy.units as u
from photutils.segmentation import SourceCatalog

unit = u.electron / u.s
catalog = SourceCatalog(data << unit, segment_map3, error=total_error << unit,
                        convolved_data=convolved_data << unit, wcs=wcs)
catalog

Our `catalog` variable is a `SourceCatalog` object. The properties of each source can be accessed using `SourceCatalog` attributes or they can be output to an Astropy `QTable` using the `to_table()` method.

Here we’ll use the `to_table()` method to generate a `QTable` of source photometry and properties. Each row in the table represents a source. The columns represent the calculated source properties. The label column corresponds to the label value in the input segmentation image.

Please see the [SourceCatalog documentation](https://photutils.readthedocs.io/en/latest/api/photutils.segmentation.SourceCatalog.html#photutils.segmentation.SourceCatalog) for a complete list of the available source properties.

In [ ]:
catalog_tbl = catalog.to_table()
catalog_tbl

Let's save this table to an ESCV file so it can be used later in the exercises.

In [ ]:
catalog_tbl.write('xdf_f160w_catalog.ecsv', overwrite=True)

Let's also save the `SegmentationImage` to a FITS file.

In [ ]:
fits.writeto('xdf_f160w_segm.fits', segment_map3.data, overwrite=True)

We can also access the source properties as attributes of the `SourceCatalog` object.

In [ ]:
catalog

In [ ]:
catalog.xcentroid

In [ ]:
catalog.eccentricity

In [ ]:
# isophotal flux
catalog.segment_flux

Next, let's plot the elliptical Kron apertures (based on the shapes of each source) for each source.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
for obj in catalog:
    obj.kron_aperture.plot(color='red', lw=2)

The `SourceCatalog` object can also be indexed or sliced to select a subset of sources.

In [ ]:
objs = catalog[0:5]  # the first 5 sources
objs

Subsets can also be created using a label number or list of labels.

In [ ]:
labels = [1, 2, 5, 7, 21]
objs = catalog.get_labels(labels)
objs

In [ ]:
objs.xcentroid

In [ ]:
objs_tbl = objs.to_table()
objs_tbl

In [ ]:
# get a single object (label=12)
obj = catalog.get_label(12)
obj

In [ ]:
obj.label

Let's plot the cutouts of the segmentation image, data, and error images for this source. These are all available as `SourceCatalog` attributes.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), ncols=3)
ax[0].imshow(obj.segment)
ax[0].set_title('Source label={} Segment'.format(obj.label))
ax[1].imshow(obj.data_ma)
ax[1].set_title('Source label={} Data'.format(obj.label))
ax[2].imshow(obj.error_ma)
ax[2].set_title('Source label={} Error'.format(obj.label));

Finally, let's plot the bounding box for this source on the data.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
obj.bbox.plot(color='red', lw=2)

## Exercise 1

The [**data/**](data) subdirectory also contains a WFC3/IR F105W image of the same field used for the preceding examples. The F105W and F160W images are pixel aligned, so sources in the F105W image are located at the same pixel positions as the F160W image (if they are visible in the F105W image).

Calculate the F105W isophotal fluxes using the source segments defined by the F160W detection image.

Hints:

* Because the images are pixel aligned, the F160W segmentation image can be directly applied to the F105W image.

* We previously saved the F160W segmentation image in a file called `xdf_f160w_segm.fits`.

* The isophotal fluxes are found in the `segment_flux` attribute of the `SourceCatalog`.

If you prefer to load the solution, uncomment the line below and run the cell twice (once to load the solution and once more to run it).

In [ ]:
# %load 04-segmentation_exercise1_solution

## Exercise 2

Calculate the $Y_{105} - H_{160}$ (F105W $-$ F160W) isophotal colors for all sources detected in the F160W image.  Which sources have the three reddest $Y_{105} - H_{160}$ colors?

The WFC3/IR F105W and F160W AB magnitude zero points are `26.2687` and `25.9463`, respectively.

Hints:

* We previously saved the F160W source catalog table in a file called `xdf_f160w_catalog.ecsv`.

If you prefer to load the solution, uncomment the line below and run the cell twice (once to load the solution and once more to run it).

In [ ]:
# %load 04-segmentation_exercise2_solution

<div class="alert alert-warning alert-block">
<h3 style='margin-top: 0;'>Learn More</h3>

The [PSF photometry](05-psf_photometry.ipynb) notebook covers:

- PSF photometry using simulated 2D Gaussian PSFs, using:
  - BasicPSFPhotometry
  - IterativelySubtractedPSFPhotometry
  - DAOPhotPSFPhotometry
</div>